
---

### **Documentação**: Script de Coleta de Notícias (Google News RSS)

#### **1. Objetivo**

Este script implementa parte do Pilar 2 (Qualidade da Comunicação) do projeto Aurum. Sua responsabilidade é automatizar a coleta de notícias financeiras recentes para cada empresa (ticker) listada no índice IBRX-100.

O script utiliza o *feed RSS do Google News* como fonte de dados, buscando menções a cada ticker nos últimos 30 dias. Os dados brutos coletados são a base para a futura análise de sentimento (NLP).

#### **2. Configuração (Input)**

O script depende de um único arquivo de entrada:

* `tickers_ibrx100_full.csv`: Um arquivo CSV que deve conter a lista completa de tickers do IBRX-100.
    * Formato esperado: O script lê a **primeira coluna** deste arquivo. Os tickers podem estar no formato `PETR4.SA` ou `PETR4`. A função `load_tickers_from_csv` remove automaticamente o sufixo `.SA` para otimizar a busca no Google News.

#### **3. Saída (Output)**

O script gera dois arquivos idênticos em conteúdo, localizados em `data/news/`:

1.  `raw_news_data.parquet`
2.  `raw_news_data.csv`

O schema (colunas) do DataFrame salvo é:

| Coluna | Tipo | Descrição |
| :--- | :--- | :--- |
| `ticker_query` | string | O ticker usado na busca (ex: `PETR4`). |
| `title` | string | O título da notícia. |
| `link` | string | O link original da notícia. |
| `published_date` | datetime | A data e hora da publicação (já convertida). |
| `source` | string | O nome do veículo de mídia (ex: "InfoMoney"). |
| `summary` | string | Um pequeno resumo ou *snippet* da notícia (HTML). |

---

In [ ]:
import os
import time
import urllib.parse

import feedparser
import pandas as pd
import tqdm

# Diretório para salvar os dados (pasta data no nível do aurum, não data_sources)
DATA_DIR = os.path.join("..", "data")
NEWS_DIR = os.path.join(DATA_DIR, "news")
os.makedirs(NEWS_DIR, exist_ok=True)

def load_tickers_from_csv(file_path: str) -> list:
    """Carrega a lista de tickers a partir de um arquivo CSV."""
    df = pd.read_csv(file_path)
    tickers = df.iloc[:, 0].dropna().astype(str).tolist()
    # Remove o sufixo '.SA' para usar na busca de notícias
    return [t.replace('.SA', '') for t in tickers]

def fetch_news_for_ticker(ticker: str):
    """Busca notícias para um ticker específico usando o RSS do Google News."""
    raw_query = f'"{ticker}" when:30d'
    search_query = urllib.parse.quote(raw_query)
    url = f"https://news.google.com/rss/search?q={search_query}&hl=pt-BR&gl=BR&ceid=BR:pt-419"
    
    feed = feedparser.parse(url)
    
    news_items = []
    for entry in feed.entries:
        news_items.append({
            'ticker_query': ticker,
            'title': entry.title,
            'link': entry.link,
            'published_date': entry.published,
            'source': entry.source.title if hasattr(entry, 'source') else 'Unknown',
            'summary': entry.summary if hasattr(entry, 'summary') else ''
        })
    return news_items


if __name__ == "__main__":
    # Caminho para o arquivo de tickers na pasta data
    tickers_csv_path = os.path.join(DATA_DIR, "tickers_ibrx100_full.csv")
    tickers = load_tickers_from_csv(tickers_csv_path)
    
    all_news = []
    
    print("Iniciando a coleta de notícias via Google News RSS...")
    for ticker in tqdm.tqdm(tickers, desc="Buscando notícias"):
        try:
            news = fetch_news_for_ticker(ticker)
            if news:
                all_news.extend(news)
            time.sleep(0.5)
        except Exception as e:
            print(f"Erro ao buscar notícias para {ticker}: {e}")

    if not all_news:
        print("\nNenhuma notícia foi coletada. Verifique a conexão ou a consulta de busca. Encerrando.")
    else:
        df_news = pd.DataFrame(all_news)
        df_news.drop_duplicates(subset=['title', 'link'], inplace=True)
        df_news['published_date'] = pd.to_datetime(df_news['published_date'], errors='coerce')
        
        # --- SALVANDO ARQUIVOS ---
        output_path_parquet = os.path.join(NEWS_DIR, "raw_news_data.parquet")
        output_path_csv = os.path.join(NEWS_DIR, "raw_news_data.csv")

        # Salva em Parquet
        df_news.to_parquet(output_path_parquet, index=False)
        
        # Salva em CSV
        df_news.to_csv(output_path_csv, index=False)
        
        print(f"\nColeta concluída. {len(df_news)} notícias únicas salvas.")
        print(f"-> {output_path_parquet}")
        print(f"-> {output_path_csv}")
        print("\nAmostra das notícias coletadas:")
        print(df_news.head())
